# Sentinel-5P Data Exploration

Data collected and processed with [s5p-tools](https://github.com/bilelomrani1/s5p-tools).

- *Study area:* Mainland France

- *Time range:* 30/04/2018 - 09/07/2019

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from itertools import product, cycle

# Ignore Runtime warnings
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

# Propagate attributes during computation
xr.set_options(keep_attrs=True);

## Data visualisation

In [ ]:
# Import NO2 database
DS = xr.open_dataset('/Users/bilelomrani/Documents/s5p-tools/processed/processed__NO2___/NO2___1-1-2019__1-1-2019.nc', chunks={'time': 100})

# Display base
DS

### Preliminaries

By default, each entry along the `time` dimension correspons to a single product. We resample the dataset so that we average together products belonging to the same day.

In [ ]:
DS = DS.resample(time='1D').mean(dim='time', skipna=None)

The dataset contains data collected between 30/04/2018 and 9/07/2019. We keep the data from 1/05/2018 to 30/06/2019 to keep full months.

In [ ]:
DS = DS.sel(time=slice('2018-05-01', '2019-06-30'))

The variable of interest is `tropospheric NO2`.

In [ ]:
VARIABLE = 'tropospheric_NO2_column_number_density'

Let's look at the column meta-data.

In [ ]:
DS[VARIABLE].attrs

We slightly reformat the labels and create a meta-data `long_name` which will be called during plotting.

In [ ]:
DS[VARIABLE].attrs['units'] = 'molec/m$^{2}$'
DS[VARIABLE].attrs['long_name'] = 'Tropospheric vertical column of NO$_2$'

LABEL = '{long_name}\n [{unit}]'.format(long_name=DS[VARIABLE].attrs['long_name'],
                                      unit=DS[VARIABLE].attrs['units'])

We create a folder for the plots.

In [ ]:
# Export directory
from os import makedirs
folder_plots = 'plots'
makedirs(folder_plots, exist_ok=True)

# Export format
FORMAT_EXPORT = 'tiff'

Some settings for plots...

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import matplotlib.font_manager as fm
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import seaborn as sns;sns.set()
import cartopy.crs as ccrs
%matplotlib inline

# Properties
COLOR_PALETTE = plt.cm.OrRd
PROJECTION = ccrs.Mercator()

# Scalebar
SCALEBAR_LENGTH = 100000   # in meters
SCALEBAR_UNIT = 'km'
NUMBER_METERS_PER_UNIT = 1000
SCALEBAR_LOCATION = 'lower left'
SCALEBAR_SEP = 3
SCALEBAR_TEXT_SIZE = 7

# Font sizes
SMALL_SIZE = 6
MEDIUM_SIZE = 9
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)   # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)   # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)   # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

### Global annual map

In [ ]:
FIGSIZE = 4

# Temporal average
data = DS.mean(dim='time')

# Figure creation
fig = plt.figure(figsize=(FIGSIZE*data.dims['longitude'] / data.dims['latitude'], FIGSIZE))
ax = plt.axes(projection=PROJECTION)
ax.outline_patch.set_edgecolor('white')

# Plot data
p = data[VARIABLE].plot(
        ax=ax,
        transform=ccrs.PlateCarree(),
        cmap=COLOR_PALETTE,
        norm=colors.LogNorm()
    )

# Add scale bar
scalebar = AnchoredSizeBar(ax.transData,
                           SCALEBAR_LENGTH, 
                           '{length} {unit}'.format(length=int(SCALEBAR_LENGTH/NUMBER_METERS_PER_UNIT), 
                                                    unit=SCALEBAR_UNIT),
                           SCALEBAR_LOCATION, 
                           pad=0.1,
                           fontproperties=fm.FontProperties(size=SCALEBAR_TEXT_SIZE),
                           sep=SCALEBAR_SEP,
                           color='black',
                           label_top=True,
                           frameon=False)
ax.add_artist(scalebar)

# Re-fit axis extent 
ax.set_extent([data.longitude.min(), data.longitude.max(), data.latitude.min(), data.latitude.max()],
              ccrs.Geodetic())

# Color bar
p.colorbar.locator = mticker.AutoLocator()
p.colorbar.formatter = mticker.ScalarFormatter(useMathText=True, useOffset=True)
p.colorbar.formatter.set_scientific(True)
p.colorbar.formatter.set_powerlimits((0,0))
p.colorbar.ax.yaxis.set_offset_position('left')
p.colorbar.ax.yaxis.labelpad = 20
p.colorbar.ax.yaxis.set_tick_params(width=1)
p.colorbar.update_ticks()

# Export
plt.savefig("{folder_plots}/france_global_NO2.{format}".format(
                folder_plots=folder_plots, format=FORMAT_EXPORT),
            bbox_inches='tight', dpi=300)

### Administrative areas maps

We generate regional maps with `regions-20180101.shp` shapefile.

In [ ]:
import cartopy.io.shapereader as shpreader
import shapely.vectorized

reader = shpreader.Reader('shp/regions-20180101.shp')
records = list(reader.records())

pd.DataFrame([entry.attributes for entry in records])

To keep the legend consistent, we need to keep in memory the min and max value accross all the territory.

In [ ]:
values = DS.mean(dim='time')[VARIABLE].values
val_min = np.nanmin(values)
val_max = np.nanmax(values)

We then iterate over the shapefile.

In [ ]:
FIGSIZE = 1.85
NROWS = 5
NCOLS = 3

SCALEBAR_LENGTH = 50000
NAME_ATTR = 'nom'

points = list(product(DS.latitude.values, DS.longitude.values))
x = [i for j, i in points]
y = [j for j, i in points]

fig, axes = plt.subplots(
    figsize=FIGSIZE * plt.figaspect(NROWS/NCOLS),
    nrows=NROWS, ncols=NCOLS, subplot_kw={'projection': PROJECTION}
)

plt.subplots_adjust(hspace=0.5, wspace=0.2)

for ax in axes.flat:
    ax.outline_patch.set_edgecolor('white')
    ax.set(aspect=1, adjustable='datalim')

index_ax = 0
for index, region in enumerate(records):
    ax = axes.flat[index_ax]
    mask = shapely.vectorized.contains(region.geometry, x, y).reshape(
        (DS.latitude.shape[0], DS.longitude.shape[0]))
    if np.any(mask):
        print("Processing {region}".format(region=region.attributes[NAME_ATTR]))
        mask_array = xr.DataArray(mask, dims=('latitude', 'longitude'),
                                      coords={'longitude': DS.longitude, 'latitude': DS.latitude})
        masked_data = DS.mean(dim='time').where(mask_array, drop=True)
        im = ax.pcolormesh(masked_data.longitude, masked_data.latitude, masked_data[VARIABLE],
                           transform=ccrs.PlateCarree(),
                           norm=colors.LogNorm(),
                           vmin=val_min, vmax=val_max,
                           cmap=COLOR_PALETTE)
        
        # Add scale bar
        scalebar = AnchoredSizeBar(ax.transData,
                       SCALEBAR_LENGTH, 
                       '{length} {unit}'.format(length=int(SCALEBAR_LENGTH/NUMBER_METERS_PER_UNIT), 
                                                unit=SCALEBAR_UNIT),
                       SCALEBAR_LOCATION, 
                       pad=-0.6,
                       sep=SCALEBAR_SEP,
                       color='black',
                       label_top=True,
                       borderpad=-0.6,
                       fontproperties=fm.FontProperties(size=SCALEBAR_TEXT_SIZE),
                       frameon=False)
        ax.add_artist(scalebar)

        ax.title.set_text(region.attributes[NAME_ATTR])
        ax.set_extent([masked_data.longitude.min(), masked_data.longitude.max(), 
                      masked_data.latitude.min(), masked_data.latitude.max()], ccrs.Geodetic())
        index_ax = index_ax + 1
    else:
        print("{region} does not intersect AOI".format(region=region.attributes[NAME_ATTR]))

# Color bar
colorbar = fig.colorbar(im, ax=axes.ravel().tolist(), aspect=40)
colorbar.locator = mticker.AutoLocator()
colorbar.formatter = mticker.ScalarFormatter(useMathText=True, useOffset=True)
colorbar.formatter.set_scientific(True)
colorbar.formatter.set_powerlimits((0,0))
colorbar.ax.yaxis.set_offset_position('left')
colorbar.ax.yaxis.labelpad = 20
colorbar.ax.yaxis.set_tick_params(width=1)
colorbar.set_label(LABEL)
colorbar.update_ticks()

plt.savefig("{folder_plots}/regions_global_NO2.{format}".format(folder_plots=folder_plots,
                                                                format=FORMAT_EXPORT),
           bbox_inches='tight', dpi=300)
plt.show()

### Monthly Boxplot

We downsample the datset per month and look at the temporal variation.

In [ ]:
boxes_months = [group[VARIABLE].values for _,group in DS.mean(dim='longitude').mean(dim='latitude').resample({'time': '1MS'})]

In [ ]:
fig = plt.figure(figsize=(6,2))
ax = plt.axes()

ax.yaxis.set_major_formatter(mticker.ScalarFormatter(useMathText=True, useOffset=True))

boxplot = sns.boxplot(data=boxes_months,
                      linewidth=0.8, width=0.5, saturation=0.5, showfliers=False, color="slategrey")
boxplot.set_xticklabels(DS.mean(dim='longitude').mean(dim='latitude').resample({'time': '1MS'}).mean('time').time.dt.strftime('%b %Y').values, rotation=30)
plt.ylabel(LABEL, labelpad=10)

plt.savefig("{folder_plots}/box_plot_NO2.{format}".format(folder_plots=folder_plots,
                                                        format=FORMAT_EXPORT), 
            bbox_inches='tight', dpi=300)

### Weekday boxplot

Grouping the entries by weekday. 

In [ ]:
# Monday=0, Sunday=6
boxes_weekday = [group[VARIABLE].values for _,group in DS.mean(dim='longitude').mean(dim='latitude').groupby('time.weekday')]

In [ ]:
WEEKDAYS_LABEL = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

fig = plt.figure(figsize=(6,2))
ax = plt.axes()
ax.yaxis.set_major_formatter(mticker.ScalarFormatter(useMathText=True, useOffset=True))

boxplot = sns.boxplot(data=boxes_weekday,
                      linewidth=0.5, width=0.5, saturation=0.5, showfliers=False, color="slategrey")
boxplot.set_xticklabels(WEEKDAYS_LABEL)
plt.ylabel(LABEL, labelpad=10)

plt.savefig("{folder_plots}/box_plot_weekdays_NO2.{format}".format(folder_plots=folder_plots,
                                                                 format=FORMAT_EXPORT),
            bbox_inches='tight', dpi=300)

### Montly maps

We resample the dataset per month;

In [ ]:
DS_month = DS.resample({'time': '1MS'}).mean(dim='time')

We keep in memory the min and max value.

In [ ]:
nat_avg_by_month_max = DS_month[VARIABLE].max(dim='longitude').max(dim='latitude')
nat_avg_by_month_min = DS_month[VARIABLE].min(dim='longitude').min(dim='latitude')

val_min = np.nanmin(nat_avg_by_month_min)
val_max = np.nanmax(nat_avg_by_month_max)

In [ ]:
FIGSIZE = 1.8
NROWS = 5
NCOLS = 3

fig, axes = plt.subplots(
    figsize=FIGSIZE * plt.figaspect(NROWS/NCOLS),
    nrows=NROWS, ncols=NCOLS, subplot_kw={'projection': PROJECTION}
)

plt.subplots_adjust(wspace=0.2, hspace=0.3)

for ax in axes.flat:
    ax.outline_patch.set_edgecolor('white')
    ax.set(aspect=1, adjustable='datalim')

index_ax = 0
for index, time in enumerate(DS_month.time):
    print('Processing {month}'.format(month=time.dt.strftime('%b %Y').values))
    ax = axes.flat[index_ax]
    data = DS_month.isel(time=index)
    im = ax.pcolormesh(data.longitude, data.latitude, data[VARIABLE],
                       transform=ccrs.PlateCarree(),
                       norm=colors.LogNorm(),
                       vmin=val_min, vmax=val_max,
                       cmap=COLOR_PALETTE)

    ax.set_title(data.time.dt.strftime('%b %Y').values)
    ax.set_extent([data.longitude.min(), data.longitude.max(), 
                  data.latitude.min(), data.latitude.max()], ccrs.Geodetic())
    index_ax = index_ax + 1

# Color bar
colorbar = fig.colorbar(im, ax=axes.ravel().tolist(), aspect=40)
colorbar.locator = mticker.AutoLocator()
colorbar.formatter = mticker.ScalarFormatter(useMathText=True, useOffset=True)
colorbar.formatter.set_scientific(True)
colorbar.formatter.set_powerlimits((19, 19))
colorbar.ax.yaxis.set_offset_position('left')
colorbar.ax.yaxis.labelpad = 20
colorbar.ax.yaxis.set_tick_params(width=1)
colorbar.set_label(LABEL)
colorbar.update_ticks()

plt.savefig("{folder_plots}/france_month_global_NO2.{format}".format(folder_plots=folder_plots,
                                                                     format=FORMAT_EXPORT),
            bbox_inches='tight', dpi=300)
plt.show()

### Maps with point of interest

In [ ]:
# Lyon, Arcelor Mittal Dunkerque, Croisement A4/A35 Vendenheim, Forêt des Landes, Pic du Midi de Bigorre
lon_points = [4.8357, 2.3212992, 7.7337068, 0.3380, 0.1411]
lat_points = [45.7640, 51.0244429, 48.6716962, 43.6034, 42.9369]

LABEL_POINTS = ['U', 'I', 'R', 'F', 'M']

In [ ]:
FIGSIZE = 4
SCALEBAR_LENGTH = 100000

# Moyenne temporelle
data = DS.mean(dim='time')

# Création du canevas
fig = plt.figure(figsize=(FIGSIZE*data.dims['longitude'] / data.dims['latitude'], FIGSIZE))
ax = plt.axes(projection=PROJECTION)
ax.outline_patch.set_edgecolor('white')

# Ajout du pcolormesh
p = data[VARIABLE].plot(
        ax=ax,
        transform=ccrs.PlateCarree(),
        cmap=COLOR_PALETTE,
        norm=colors.LogNorm()
    )

# Add points labels
droite = 9.5
gauche = -6
milieu = DS.longitude.mean().values

for i,(x,y) in enumerate(zip(lon_points,lat_points)):
    if x < milieu:
        plt.plot([x, gauche],[y, y], '-', color='black', linewidth=0.3, transform=ccrs.PlateCarree())
        plt.text(gauche-0.5, y-0.2, LABEL_POINTS[i], transform=ccrs.PlateCarree(), fontsize=7)
    else:
        plt.plot([x, droite],[y, y], '-', color='black', linewidth=0.3, transform=ccrs.PlateCarree())
        plt.text(droite+0.4, y-0.2, LABEL_POINTS[i], transform=ccrs.PlateCarree(), fontsize=7)

# Add point
s = sns.scatterplot(lon_points, lat_points, transform=ccrs.PlateCarree(),
               color=".2", s=10, linewidth=0.5)

# Add scale bar
scalebar = AnchoredSizeBar(ax.transData,
                           SCALEBAR_LENGTH, 
                           '{length} {unit}'.format(length=int(SCALEBAR_LENGTH/NUMBER_METERS_PER_UNIT), 
                                                    unit=SCALEBAR_UNIT),
                           SCALEBAR_LOCATION, 
                           pad=0.1,
                           fontproperties=fm.FontProperties(size=SCALEBAR_TEXT_SIZE),
                           sep=SCALEBAR_SEP,
                           color='black',
                           label_top=True,
                           frameon=False)
ax.add_artist(scalebar)

# Re-fit axis extent 
ax.set_extent([data.longitude.min(), data.longitude.max(), data.latitude.min(), data.latitude.max()],
              ccrs.Geodetic())

# Color bar
p.colorbar.locator = mticker.AutoLocator()
p.colorbar.formatter = mticker.ScalarFormatter(useMathText=True, useOffset=True)
p.colorbar.formatter.set_scientific(True)
p.colorbar.ax.yaxis.set_offset_position('left')
p.colorbar.ax.yaxis.labelpad = 20
p.colorbar.ax.yaxis.set_tick_params(width=1)
p.colorbar.update_ticks()

# Export
plt.savefig("{folder_plots}/dotted_france_global_NO2.{format}".format(folder_plots=folder_plots, format=FORMAT_EXPORT),
            bbox_inches='tight', dpi=300)

### PI evolution

In [ ]:
fig = plt.figure(figsize=(10,4))
ax = plt.axes()

# format the ticks
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
ax.xaxis.set_minor_locator(mdates.DayLocator())
ax.yaxis.set_major_formatter(mticker.ScalarFormatter(useMathText=True, useOffset=True))

for index,(lon, lat) in enumerate(zip(lon_points, lat_points)):

    DS.resample({'time': '1MS'}).mean(dim='time')[VARIABLE].sel(latitude=lat, longitude=lon, method='nearest').plot(
        label=LABEL_POINTS[index], ax=ax, linewidth=1)

# Change le style pour impression-friendly
for l, ms in zip(ax.lines, cycle('ox+>^*')):
    l.set_marker(ms)

plt.xlabel('')
plt.ylabel(LABEL)
plt.title('')
    
plt.legend()
plt.xticks(rotation=30)

plt.savefig("{folder_plots}/evol_points_interet_NO2.{format}".format(folder_plots=folder_plots, format=FORMAT_EXPORT),
            bbox_inches='tight', dpi=300)